In [1]:
import pandas as pd
import numpy as np


In [2]:
# Charger les fichiers
freq = pd.read_csv("../data/raw/fremtpl2/freMTPL2freq.csv")
sev = pd.read_csv("../data/raw/fremtpl2/freMTPL2sev.csv")

# 1️⃣ Lignes avec ClaimNb = 0
freq_no_claim = freq[freq["ClaimNb"] == 0].copy()
freq_no_claim["ClaimAmount"] = 0

# 2️⃣ Lignes avec ClaimNb > 0 ET présentes dans sev
freq_with_claim = freq[freq["ClaimNb"] > 0].copy()
freq_with_claim = freq_with_claim.merge(sev, on="IDpol", how="inner")  # inner join pour ne garder que ceux présents dans sev

# 3️⃣ Concaténer les deux
policy_master = pd.concat([freq_no_claim, freq_with_claim], ignore_index=True)

# 4️⃣ Vérification rapide
print(policy_master.shape)
print(policy_master["ClaimAmount"].head(10))
print(policy_master["ClaimNb"].value_counts())
policy_master.head()
# 5️⃣ Sauvegarde (optionnel)
#policy_master.to_csv("data/processed/policy_master.csv", index=False)


(670397, 13)
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: ClaimAmount, dtype: float64
ClaimNb
0     643953
1      23570
2       2597
3        186
11        22
4         20
16        16
5         10
9          9
8          8
6          6
Name: count, dtype: int64


,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount
0,24952.0,0,0.005464,A,6,15,50,50,B2,Diesel,29,R24,0.0
1,24953.0,0,0.670000,A,6,15,50,50,B2,Diesel,29,R24,0.0
2,24955.0,0,0.890000,A,6,8,30,68,B1,Diesel,48,R53,0.0
3,24956.0,0,0.100000,A,6,8,30,57,B1,Diesel,48,R53,0.0
4,24958.0,0,0.200000,B,7,4,44,50,B14,Diesel,56,R24,0.0


In [3]:
policy_master["claim_occurred"] = (policy_master["ClaimNb"] > 0).astype(int)
policy_master.groupby("claim_occurred")["ClaimAmount"].describe()

,count,mean,std,min,25%,50%,75%,max
claim_occurred,,,,,,,,
0,643953.0,0.000000,0.000000,0.0,0.0000,0.0,0.000,0.00
1,26444.0,2265.512649,29371.031113,1.0,685.9925,1172.0,1212.385,4075400.56


In [4]:
import numpy as np

rng = np.random.default_rng(42)

df = policy_master.copy()

# ----------------------------
# 0) Years active (maturity)
# ----------------------------
# Exposure is usually in years in freMTPL2 (often <= 1). We'll keep it, and clip.
df["years_active"] = df["Exposure"].astype(float).clip(0.05, 5.0)

# Create a maturity score in [0,1] (simple scaling)
# If Exposure mostly in [0,1], this is basically maturity within first year.
df["maturity"] = (df["years_active"] / df["years_active"].quantile(0.95)).clip(0, 1)

# ----------------------------
# 1) Shop area (continuous)
# ----------------------------
base_area = df["VehPower"].astype(float) * 10
# Add controlled noise + a mild maturity boost (mature shops tend to be larger or better organized)
df["shop_area_m2"] = (
    base_area
    * rng.lognormal(mean=0, sigma=0.15, size=len(df))
    * (0.95 + 0.20 * df["maturity"])
).clip(8, 220)

# ----------------------------
# 2) Activity type (fuzzy borders + maturity)
# ----------------------------
# Mature businesses slightly more likely to be "pharmacy/clothing" for same size.
def activity_from_area_and_maturity(area, m):
    # area drives the base category; maturity nudges upward slightly
    if area < 45:
        return rng.choice(["kiosk", "cafe"], p=[0.65 - 0.15*m, 0.35 + 0.15*m])
    elif area < 80:
        return rng.choice(["cafe", "grocery"], p=[0.35 - 0.10*m, 0.65 + 0.10*m])
    elif area < 120:
        return rng.choice(["grocery", "clothing"], p=[0.50 - 0.20*m, 0.50 + 0.20*m])
    else:
        return rng.choice(["clothing", "pharmacy"], p=[0.40 - 0.15*m, 0.60 + 0.15*m])

df["activity_type"] = [
    activity_from_area_and_maturity(a, m)
    for a, m in zip(df["shop_area_m2"].values, df["maturity"].values)
]

# ----------------------------
# 3) Assets value (area × activity × maturity + noise)
# ----------------------------
activity_assets_mult = {
    "kiosk": 0.6,
    "cafe": 0.8,
    "grocery": 1.0,
    "clothing": 1.1,
    "pharmacy": 1.4,
}

base_assets = df["shop_area_m2"] * 700 * df["activity_type"].map(activity_assets_mult).astype(float)

# maturity increases assets (inventory + equipment) moderately
df["assets_value_tnd"] = (
    base_assets
    * (0.85 + 0.40 * df["maturity"])
    * rng.lognormal(0, 0.18, len(df))
).clip(500, 600000)

# ----------------------------
# 4) Revenue monthly (assets × turnover_rate + density + maturity)
# ----------------------------
density_norm = np.log1p(df["Density"].astype(float)) / np.log1p(df["Density"].astype(float).max())

# turnover rate depends on activity (simple and explainable)
turnover_rate = {
    "kiosk": 0.11,
    "cafe": 0.10,
    "grocery": 0.12,
    "clothing": 0.09,
    "pharmacy": 0.13,
}
base_turn = df["activity_type"].map(turnover_rate).astype(float).values

# add small random variation around base turnover
turn = base_turn * rng.uniform(0.9, 1.1, len(df))

df["revenue_monthly_tnd"] = (
    df["assets_value_tnd"].values
    * turn
    * (0.90 + 0.20 * density_norm.values)     # density effect ±10%
    * (0.90 + 0.20 * df["maturity"].values)   # maturity effect ±10%
).clip(250, 150000)

# ----------------------------
# 5) Revenue bucket (quantiles => robust)
# ----------------------------
df["revenue_bucket"] = pd.qcut(df["revenue_monthly_tnd"], q=3, labels=["low", "medium", "high"]).astype(str)

# ----------------------------
# 6) Open at night (activity + slight density)
# ----------------------------
base_night = {"kiosk": 0.35, "cafe": 0.70, "grocery": 0.40, "clothing": 0.15, "pharmacy": 0.10}
p_night = df["activity_type"].map(base_night).astype(float).values
p_night = np.clip(p_night + 0.05*(density_norm.values - 0.5), 0.05, 0.85)
df["open_at_night"] = rng.binomial(1, p_night, len(df)).astype(bool)

# ----------------------------
# 7) Security (assets + maturity, very simple)
# ----------------------------
assets = df["assets_value_tnd"].values
m = df["maturity"].values

# alarm probability increases with assets and maturity
p_alarm = 1 / (1 + np.exp(-(-3.0 + 0.00006*assets + 1.2*m)))
p_cam   = 1 / (1 + np.exp(-(-3.8 + 0.00006*assets + 1.0*m)))

df["security_alarm"]  = rng.binomial(1, np.clip(p_alarm, 0.02, 0.95)).astype(bool)
df["security_camera"] = rng.binomial(1, np.clip(p_cam,   0.01, 0.90)).astype(bool)

# fire extinguisher: mostly activity + maturity
base_fire = {"kiosk": 0.20, "cafe": 0.35, "grocery": 0.30, "clothing": 0.25, "pharmacy": 0.55}
p_fire = df["activity_type"].map(base_fire).astype(float).values
p_fire = np.clip(p_fire + 0.20*m, 0.05, 0.90)
df["fire_extinguisher"] = rng.binomial(1, p_fire, len(df)).astype(bool)

# ----------------------------
# 😎 Labels (keep unchanged)
# ----------------------------
df["claim_occurred"] = (df["ClaimNb"] > 0).astype(int)
df["claim_cost_tnd"] = df["ClaimAmount"].astype(float)

# ----------------------------
# 9) Keep only the useful columns (optional)
# ----------------------------
keep_cols = [
    "policy_id" if "policy_id" in df.columns else "IDpol",
    "governorate" if "governorate" in df.columns else "Area",
    "years_active",
    "activity_type",
    "shop_area_m2",
    "assets_value_tnd",
    "revenue_monthly_tnd",
    "revenue_bucket",
    "open_at_night",
    "security_alarm",
    "security_camera",
    "fire_extinguisher",
    "claim_occurred",
    "claim_cost_tnd"
]

keep_cols = [c for c in keep_cols if c in df.columns]
business_df = df[keep_cols].copy()

business_df.head()

,IDpol,Area,years_active,activity_type,shop_area_m2,assets_value_tnd,revenue_monthly_tnd,revenue_bucket,open_at_night,security_alarm,security_camera,fire_extinguisher,claim_occurred,claim_cost_tnd
0,24952.0,A,0.05,grocery,60.293851,41636.955278,4749.817913,medium,False,False,True,True,0,0.0
1,24953.0,A,0.67,grocery,55.645702,51021.477592,6653.990820,high,False,False,False,False,0,0.0
2,24955.0,A,0.89,grocery,75.743936,59667.863956,8224.172273,high,False,True,True,True,0,0.0
3,24956.0,A,0.10,grocery,67.018591,51336.243377,6071.250481,medium,False,True,False,True,0,0.0
4,24958.0,B,0.20,grocery,51.717159,34968.809587,3748.563655,low,False,False,False,True,0,0.0


In [5]:
business_df.to_csv("../data/processed/policy_master_business.csv", index=False)


In [6]:
import pandas as pd

# 1. Charger les données INS
df_ins = pd.read_excel('../data/raw/ins/Nombre Ménage Gouv.xlsx', engine='openpyxl')

# 2. Garder uniquement les colonnes utiles et les renommer proprement
df_ins = df_ins[['Gouvernorat', 'Valeur']].copy()
df_ins.columns = ['governorate', 'nb_menages']

# 3. NETTOYAGE CRUCIAL (pour que 'Tunis' matche avec 'Tunis ')
df_ins['governorate'] = df_ins['governorate'].astype(str).str.strip()

# 4. Ajouter la superficie pour calculer la densité
superficies = {
    'Tunis': 288, 'Ariana': 482, 'Ben Arous': 761, 'Manouba': 1137,
    'Nabeul': 2788, 'Zaghouan': 2768, 'Bizerte': 3685, 'Béja': 3558, # Note l'accent sur Béja
    'Jendouba': 3102, 'Le Kef': 4965, 'Siliana': 4642, 'Kairouan': 6712,
    'Kasserine': 8065, 'Sidi Bouzid': 6994, 'Sousse': 2621, 'Monastir': 1019,
    'Mahdia': 2966, 'Sfax': 7545, 'Gafsa': 8990, 'Tozeur': 4719,
    'Kebili': 22084, 'Gabès': 7175, 'Médenine': 8588, 'Tataouine': 38889
}
df_surf = pd.DataFrame(list(superficies.items()), columns=['governorate', 'area_km2'])

# 5. Fusionner et calculer la densité
df_ins = df_ins.merge(df_surf, on='governorate', how='inner')
df_ins['household_density'] = df_ins['nb_menages'] / df_ins['area_km2']

# 6. Maintenant, fusionne avec ton dataset principal (Small Business)
# df_final = df_votre_data.merge(df_ins[['governorate', 'household_density']], on='governorate', how='left')

print("Succès ! Voici un aperçu de ta densité par gouvernorat :")
print(df_ins[['governorate', 'household_density']].sort_values(by='household_density', ascending=False).head())

Succès ! Voici un aperçu de ta densité par gouvernorat :
   governorate  household_density
0        Tunis        1251.152778
1       Ariana         430.039419
2    Ben Arous         288.176084
11    Monastir         166.420020
3      Manouba         106.501319


In [7]:
df_ins.to_csv("../data/processed/policy_master_ins.csv", index=False)

In [8]:
# Données de superficie (km2) pour le Grand Tunis, Sousse et Sfax
data_geo = {
    "Tunis": {"area": 288, "poi_count": 30562},      # Ta valeur reçue
    "Ariana": {"area": 482, "poi_count": 2278},         # À remplir avec Overpass
    "Ben Arous": {"area": 761, "poi_count": 2362},      # À remplir avec Overpass
    "Manouba": {"area": 1137, "poi_count": 796},       # À remplir avec Overpass
    "Sfax": {"area": 7545, "poi_count": 2557},          # À remplir avec Overpass
    "Sousse": {"area": 2621, "poi_count": 1012}         # À remplir avec Overpass
}

# Calcul automatique de la densité
for gov, values in data_geo.items():
    if values["poi_count"] > 0:
        values["poi_density"] = values["poi_count"] / values["area"]
    else:
        values["poi_density"] = 0

print(data_geo)

{'Tunis': {'area': 288, 'poi_count': 30562, 'poi_density': 106.11805555555556}, 'Ariana': {'area': 482, 'poi_count': 2278, 'poi_density': 4.726141078838174}, 'Ben Arous': {'area': 761, 'poi_count': 2362, 'poi_density': 3.1038107752956634}, 'Manouba': {'area': 1137, 'poi_count': 796, 'poi_density': 0.7000879507475813}, 'Sfax': {'area': 7545, 'poi_count': 2557, 'poi_density': 0.3388999337309476}, 'Sousse': {'area': 2621, 'poi_count': 1012, 'poi_density': 0.3861121709271271}}


In [9]:
import numpy as np

# On applique le log pour lisser l'écart entre Tunis et le reste
for gov in data_geo:
    data_geo[gov]['poi_density_log'] = np.log1p(data_geo[gov]['poi_density'])

# Tunis passera de 19.7 à ~3.0
# Manouba passera de 0.06 à ~0.05
# Les écarts deviennent "traitables" par une IA.
for city in data_geo:
    data_geo[city]['poi_density_log'] = float(data_geo[city]['poi_density_log'])
print(data_geo)

{'Tunis': {'area': 288, 'poi_count': 30562, 'poi_density': 106.11805555555556, 'poi_density_log': 4.673931549197588}, 'Ariana': {'area': 482, 'poi_count': 2278, 'poi_density': 4.726141078838174, 'poi_density_log': 1.7450418446605953}, 'Ben Arous': {'area': 761, 'poi_count': 2362, 'poi_density': 3.1038107752956634, 'poi_density_log': 1.4119159994213928}, 'Manouba': {'area': 1137, 'poi_count': 796, 'poi_density': 0.7000879507475813, 'poi_density_log': 0.5306799854577948}, 'Sfax': {'area': 7545, 'poi_count': 2557, 'poi_density': 0.3388999337309476, 'poi_density_log': 0.29184833182604863}, 'Sousse': {'area': 2621, 'poi_count': 1012, 'poi_density': 0.3861121709271271, 'poi_density_log': 0.3265028288977956}}


In [10]:
import pandas as pd

# Read CSV data (only once)
df = pd.read_csv("../data/processed/policy_master_ins.csv")

# Define POI data dictionary
# You need to populate this with your actual POI data

# Filter to keep only governorates in the POI dictionary
governorates_to_keep = list(data_geo.keys())
df_filtered = df[df['governorate'].isin(governorates_to_keep)].copy()

# Add POI columns
df_filtered['poi_count'] = df_filtered['governorate'].map(
    lambda x: data_geo[x]['poi_count']
)
df_filtered['poi_density'] = df_filtered['governorate'].map(
    lambda x: data_geo[x]['poi_density_log']
)

# Display the result
print("Filtered DataFrame with POI data:")
print(df_filtered.head())

# Save to CSV
df_filtered.to_csv('../data/processed/filtered_governorates_with_poi.csv', index=False)
print("\nData saved to 'filtered_governorates_with_poi.csv'")

Filtered DataFrame with POI data:
   governorate  nb_menages  area_km2  household_density  poi_count  \
0        Tunis      360332       288        1251.152778      30562   
1       Ariana      207279       482         430.039419       2278   
2    Ben Arous      219302       761         288.176084       2362   
3      Manouba      121092      1137         106.501319        796   
10      Sousse      228513      2621          87.185425       1012   

    poi_density  
0      4.673932  
1      1.745042  
2      1.411916  
3      0.530680  
10     0.326503  

Data saved to 'filtered_governorates_with_poi.csv'


In [11]:
import pandas as pd
import numpy as np

# ====================================================================
# AREA TO GOVERNORATE MAPPING
# ====================================================================
# Logic: Map Area codes (A-F) to Tunisian governorates based on density
# 
# Rationale:
# - Area codes in insurance datasets typically reflect urbanization/risk
# - We map high-density areas to high-density governorates
# - This preserves the risk signal while making data Tunisia-specific
# ====================================================================

# Load your business dataset
business_df = pd.read_csv("../data/processed/policy_master_business.csv")

# Load governorate statistics
gov_stats = pd.read_csv("../data/processed/filtered_governorates_with_poi.csv")

# Sort governorates by household density (descending)
gov_stats_sorted = gov_stats.sort_values('household_density', ascending=False)

print("Governorates ranked by density:")
print(gov_stats_sorted[['governorate', 'household_density', 'poi_density']])
print("\n")

# ====================================================================
# DETERMINISTIC MAPPING TABLE
# ====================================================================
# Based on density ranking:
# A, B → Highest density (Tunis, Ariana)
# C, D → Medium density (Ben Arous, Sousse)
# E, F → Lower density (Sfax, Manouba)
# ====================================================================

area_to_gov_mapping = {
    'A': 'Tunis',      # Highest density (1251)
    'B': 'Ariana',     # High density (430)
    'C': 'Ben Arous',  # Medium-high (288)
    'D': 'Manouba',     # Medium (87)
    'E': 'Sousse',       # Medium-low (40)
    'F': 'Sfax'     # Lowest density (106)
}

print("Area to Governorate Mapping:")
for area, gov in area_to_gov_mapping.items():
    density = gov_stats[gov_stats['governorate'] == gov]['household_density'].values[0]
    print(f"  Area {area} → {gov} (density: {density:.1f} households/km²)")
print("\n")

# ====================================================================
# APPLY MAPPING
# ====================================================================

# Map Area codes to governorates
business_df['governorate'] = business_df['Area'].map(area_to_gov_mapping)

# Merge with governorate statistics to get density and POI data
business_df = business_df.merge(
    gov_stats[['governorate', 'household_density', 'poi_count', 'poi_density']], 
    on='governorate', 
    how='left'
)

# Drop the original Area column (optional, keep if you want traceability)
# business_df = business_df.drop(columns=['Area'])

# Rename columns for clarity
business_df.rename(columns={
    'household_density': 'density_per_km2',
    'poi_density': 'poi_per_km2'
}, inplace=True)

# ====================================================================
# REORDER COLUMNS FOR CLARITY
# ====================================================================
column_order = [
    'IDpol',
    'governorate',
    'density_per_km2',
    'poi_per_km2',
    'years_active',
    'activity_type',
    'shop_area_m2',
    'assets_value_tnd',
    'revenue_monthly_tnd',
    'revenue_bucket',
    'open_at_night',
    'security_alarm',
    'security_camera',
    'fire_extinguisher',
    'claim_occurred',
    'claim_cost_tnd'
]

# Keep only columns that exist
column_order = [col for col in column_order if col in business_df.columns]
business_df = business_df[column_order]

# ====================================================================
# VALIDATION & STATISTICS
# ====================================================================
print("Mapping Distribution:")
print(business_df['governorate'].value_counts().sort_index())
print("\n")

print("Sample of mapped data:")
print(business_df.head(10))
print("\n")

# Check for any unmapped areas
unmapped = business_df[business_df['governorate'].isna()]
if len(unmapped) > 0:
    print(f"⚠️  Warning: {len(unmapped)} rows with unmapped areas")
    print(unmapped['Area'].value_counts())
else:
    print("✅ All areas successfully mapped!")

# ====================================================================
# SAVE FINAL DATASET
# ====================================================================
business_df.to_csv('../data/processed/business_df_with_governorate.csv', index=False)
print("\n✅ Final dataset saved to 'business_df_with_governorate.csv'")



Governorates ranked by density:
  governorate  household_density  poi_density
0       Tunis        1251.152778     4.673932
1      Ariana         430.039419     1.745042
2   Ben Arous         288.176084     1.411916
3     Manouba         106.501319     0.530680
4      Sousse          87.185425     0.326503
5        Sfax          40.366070     0.291848


Area to Governorate Mapping:
  Area A → Tunis (density: 1251.2 households/km²)
  Area B → Ariana (density: 430.0 households/km²)
  Area C → Ben Arous (density: 288.2 households/km²)
  Area D → Manouba (density: 106.5 households/km²)
  Area E → Sousse (density: 87.2 households/km²)
  Area F → Sfax (density: 40.4 households/km²)


Mapping Distribution:
governorate
Ariana        74462
Ben Arous    189617
Manouba      150162
Sfax          17673
Sousse       135993
Tunis        102490
Name: count, dtype: int64


Sample of mapped data:
     IDpol governorate  density_per_km2  poi_per_km2  years_active  \
0  24952.0       Tunis      1251.15277